In [28]:
# Import libararies
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [29]:
# implement a function to Enter dates and Search for the table
def scrape_dld_transaction_data_to_csv():
    # Initialize Chrome WebDriver
    driver = webdriver.Chrome()
    driver.get("https://dubailand.gov.ae/en/open-data/real-estate-data/#/")
    driver.implicitly_wait(10)  # Wait for elements to appear

    # Execute JavaScript to set the From Date and To Date to desired values
    start_date = "06/01/2024"
    end_date = "08/01/2024"
    script = f'''
        document.getElementById('transaction_pFromDate').value = '{start_date}';
        document.getElementById('transaction_pToDate').value = '{end_date}';
        document.querySelector('button.btn.btn_1.btn_style1').click();
    '''
    driver.execute_script(script)
    print("Search button clicked")

    # Wait for the table to load
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "transactionGrid_info")))
    print("Table loaded")

    # Execute JavaScript to select '100' from the dropdown menu
    select_script = '''
        var dropdown = document.querySelector('select[name="transactionGrid_length"]');
        dropdown.value = '100';
        dropdown.dispatchEvent(new Event('change'));
        
    '''
    driver.execute_script(select_script)  

    # Close the WebDriver session
    driver.quit()

# Scrape data and save to CSV
scrape_dld_transaction_data_to_csv()


Search button clicked
Table loaded


In [31]:
# implement a function to Enter dates and Search for the table
def scrape_dld_transaction_data_to_csv():
    # Initialize Chrome WebDriver
    driver = webdriver.Chrome()
    driver.get("https://dubailand.gov.ae/en/open-data/real-estate-data/#/")
    driver.implicitly_wait(10)  # Wait for elements to appear

    # Execute JavaScript to set the From Date and To Date to desired values
    start_date = "20/01/2024"
    end_date = "05/02/2024"
    script = f'''
        document.getElementById('transaction_pFromDate').value = '{start_date}';
        document.getElementById('transaction_pToDate').value = '{end_date}';
        document.querySelector('button.btn.btn_1.btn_style1').click();
    '''
    driver.execute_script(script)
    print("Search button clicked")

    # Wait for the table to load
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "transactionGrid_info")))
    print("Table loaded")

    # Execute JavaScript to select '100' from the dropdown menu
    select_script = '''
        var dropdown = document.querySelector('select[name="transactionGrid_length"]');
        dropdown.value = '100';
        dropdown.dispatchEvent(new Event('change'));
        
    '''
    driver.execute_script(select_script) 

    # Extract data from all pages
    all_data = []
    while True:

        WebDriverWait(driver, 3).until_not(EC.visibility_of_element_located((By.ID, 'transactionGrid_processing')))
        # Extract data from the current page
        rows = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#transactionGrid tbody tr')))
        for row in rows:
            row_data = [td.text for td in row.find_elements(By.TAG_NAME, 'td')]
            all_data.append(row_data)

        # Check if there is a next page
        next_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, 'transactionGrid_next')))
        if 'disabled' in next_button.get_attribute('class'):
            break  # Exit the loop if there's no next page

        # Scroll to the next button
        driver.execute_script("arguments[0].scrollIntoView();", next_button)

        # Click on the next button to go to the next page
        driver.execute_script("arguments[0].click();", next_button)

        # Wait for the table to refresh
        WebDriverWait(driver, 3).until(EC.staleness_of(rows[0]))  
        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "transactionGrid_info")))

    # Close the WebDriver session
    driver.quit()

    # If there's no data, exit the function
    if not all_data:
        print("No data available in table")
        return

    # Convert the data to DataFrame
    df = pd.DataFrame(all_data, columns=[
        'Transaction Number', 'Transaction Date', 'Transaction Type', 'Transaction sub type',
        'Registration type', 'Is Free Hold?', 'Usage', 'Area', 'Property Type', 'Property Sub Type',
        'Amount', 'Transaction Size (sq.m)', 'Property Size (sq.m)', 'Room(s)', 'Parking',
        'Nearest Metro', 'Nearest Mall', 'Nearest Landmark', 'No. of Buyer', 'No. of Seller',
        'Master Project', 'Project'
    ])

    # Export DataFrame to CSV
    df.to_csv('dld_transactions.csv', index=False)
    print("Data exported to CSV")


# Scrape data and save to CSV
scrape_dld_transaction_data_to_csv()


Search button clicked
Table loaded
Data exported to CSV
